1st goal: ETL process completely in the cloud and upload a DataFrame to an RDS instance. 

2nd goal: Use PySpark or SQL to perform a statistical analysis of selected data.

In [4]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad

In [5]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-02 14:07:05--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.52MB/s    in 0.2s    

2021-10-02 14:07:05 (4.52 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [6]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonSoftwareReviews").getOrCreate()

In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [8]:
df.count()

341931

In [9]:
software_df = df.dropna()
software_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [10]:
software_df.count()

341913

In [14]:
software_df.dropDuplicates().show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   38242199|R2LCT3GZXQF41Y|B0055A7RS8|      79858647|Individual Softwa...|        Software|          5|            0|          0|   N|                Y|Like the contacts...|Upgraded from Del...| 2015-08-31|
|         US|   22289015| R3VS4ADA5CN7S|B00CLVCY2I|      61914058|ESET NOD32 Antivi...|        Software|          5|    

In [15]:
software_df.dropDuplicates().count()

341913

In [16]:
software_df = software_df.dropDuplicates()

In [17]:
software_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [18]:
from pyspark.sql.types import IntegerType
software_df = software_df.withColumn("customer_id",software_df["customer_id"].cast(IntegerType()))
software_df = software_df.withColumn("product_parent",software_df["product_parent"].cast(IntegerType()))
software_df = software_df.withColumn("star_rating",software_df["star_rating"].cast(IntegerType()))
software_df = software_df.withColumn("helpful_votes",software_df["helpful_votes"].cast(IntegerType()))
software_df = software_df.withColumn("total_votes",software_df["total_votes"].cast(IntegerType()))

In [19]:

print(software_df.select("customer_id").dtypes)
print(software_df.select("product_parent").dtypes)
print(software_df.select("star_rating").dtypes)
print(software_df.select("helpful_votes").dtypes)
print(software_df.select("total_votes").dtypes)

[('customer_id', 'int')]
[('product_parent', 'int')]
[('star_rating', 'int')]
[('helpful_votes', 'int')]
[('total_votes', 'int')]


In [20]:
software_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



Trnasforming dataset to fit tables in the Schema File

In [22]:
from pyspark.sql.functions import to_date
review_id = software_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2LCT3GZXQF41Y|   38242199|B0055A7RS8|      79858647| 2015-08-31|
| R3VS4ADA5CN7S|   22289015|B00CLVCY2I|      61914058| 2015-08-31|
|R3OVT9X8XPG7UK|   14733045|B002N5YI44|     970535244| 2015-08-29|
|R2314J99M3WLJH|   18753915|B000E28UT2|     642183116| 2015-08-27|
| RR085KKEB9JAW|   18048088|B003SEVL8U|     386618608| 2015-08-26|
|R16KZEFWC2YHAS|   11105771|B00MYXTCIM|     682709709| 2015-08-25|
| ROUBDN5Z680QP|   52592720|B00DPHOV0A|     148965349| 2015-08-25|
|R3HEY3HINUWX0H|   46164973|B00E6LJBUO|     915569085| 2015-08-24|
|R3MFIZ1YAS9R44|   49531926|B00MA5TUGI|     925385625| 2015-08-19|
| RG9ZJWRAQIX7Z|   52960176|B009SPN9O2|     189849256| 2015-08-17|
|R1P1PQB5PJ2FGG|   37835716|B002SRNS7A|     213462427| 2015-08-15|
| RNP31X4PWX910|   11733416|B00LC9USLO|     962800663| 2015-08

In [23]:
from pyspark import SparkFiles
products = software_df.select(["product_id", "product_title"])
products = products.drop_duplicates(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000XYP24M|Nova Development ...|
|B001Q24UAM|Play! The Ultimat...|
|B003L51CZ8|Sony Vegas Movie ...|
|B002JB3BC2|Anti-Virus & Anti...|
|B000WM1Z64|Microsoft Visual ...|
|B004625PV2|My Golf Game feat...|
|B00003069T|ClueFinders 5th G...|
|B000F7VRIC|Bibble Pro 4.9 (W...|
|B00NNU0C3Y|MAGIX Music Maker...|
|B004TLLPZU|The Next Big Thin...|
|B000EJ9W0Q|Adobe Photoshop E...|
|B00008NJEG|Learn To Speak Fr...|
|B000ROD3W6|Apple Aperture 2....|
|B000NIBB5W|7007 Slots: Virtu...|
|B0009STM5M|Jumpstart Advance...|
|B00005QI6M|Cook'n with Taste...|
|B00005QVPY|Adobe Illustrator...|
|B000050GEU|Quicken TurboTax ...|
|B00CHZG1FE|Image Line FL Stu...|
|B000TFI0AI|Rosetta Stone V2:...|
+----------+--------------------+
only showing top 20 rows



In [24]:
from pyspark import SparkFiles
# DataFrame
customers = software_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    7455053|             9|
|   19365223|             1|
|   42863979|             1|
|   28424356|             1|
|   17533812|             1|
|   31337977|             1|
|   16135059|             1|
|   39360357|             1|
|   16217101|             1|
|    7401551|             2|
|   45702941|             1|
|   51466865|             1|
|   46426203|             1|
|   50874998|             8|
|   51144753|             1|
|   31862546|             2|
|   50947785|             1|
|   18785199|             4|
|   48081045|             3|
|    2309648|             1|
+-----------+--------------+
only showing top 20 rows



In [25]:
from pyspark import SparkFiles
vine_table = software_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2LCT3GZXQF41Y|          5|            0|          0|   N|
| R3VS4ADA5CN7S|          5|            0|          0|   N|
|R3OVT9X8XPG7UK|          2|            0|          0|   N|
|R2314J99M3WLJH|          5|            3|          3|   N|
| RR085KKEB9JAW|          1|            0|          0|   N|
|R16KZEFWC2YHAS|          3|            0|          0|   N|
| ROUBDN5Z680QP|          5|            0|          0|   N|
|R3HEY3HINUWX0H|          4|            0|          0|   N|
|R3MFIZ1YAS9R44|          4|            0|          0|   N|
| RG9ZJWRAQIX7Z|          4|            0|          0|   N|
|R1P1PQB5PJ2FGG|          5|            0|          0|   N|
| RNP31X4PWX910|          5|            0|          0|   N|
|R2LYZHT7DKPJ1Y|          5|            0|          0|   N|
|R3CQ2N80DL074E|          5|            

In [31]:
mode = 'append'
jdbc_url = "jdbc:postgresql://mypostgresdb.ceomfsf1omq9.us-east-2.rds.amazonaws.com:5432/my_data_class_db_software"
config = {
    'user':'postgres',
    'password':'********',
    'driver':"org.postgresql.Driver"
}

In [32]:
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [33]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [34]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [35]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored